In [1]:
#We import the necessary libraries.
%matplotlib inline

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import optimize
from importlib import reload

In [3]:
#We first import the NEID data.
df = pd.read_csv("TIC 258804746_rv_unbin_newest.csv")

df

,bjd,bjdnonweighted,deltamin,rv,e_rv,pre_rv,pre_e_rv,sn18,sn102,exptime,...,NaD2_ind_e,PaD_ind_v,PaD_ind_e,NaNIR_ind_v,NaNIR_ind_e,rvccfsum,rvccfsumerr,rvccfmod,rvccfmoderr,driftfun
0,2.460662e+06,2.460662e+06,0.173601,-8.980639,2.618923,NaN,NaN,13.223364,28.454166,299.842613,...,0.004771,0.862367,0.029393,0.178822,0.003329,12.628763,0.003462,12.630490,0.003437,dailymodel0
1,2.460662e+06,2.460662e+06,-0.114278,-21.921024,3.141299,NaN,NaN,10.862042,24.040826,299.837112,...,0.005885,0.885610,0.036821,0.175247,0.003900,12.607195,0.003987,12.608372,0.003944,dailymodel0
2,2.460664e+06,2.460664e+06,-0.096900,-1.938046,2.359089,NaN,NaN,14.915557,31.877791,299.796731,...,0.004140,0.857568,0.024962,0.177756,0.002934,12.631427,0.003232,12.631962,0.003183,dailymodel0
3,2.460665e+06,2.460665e+06,-0.003376,6.412131,1.850221,NaN,NaN,19.394139,38.013901,299.791047,...,0.003100,0.853720,0.019657,0.181766,0.002430,12.638462,0.002631,12.639689,0.002613,dailymodel0
4,2.460666e+06,2.460666e+06,0.016561,9.844131,1.807651,NaN,NaN,20.013067,38.943889,299.809944,...,0.003036,0.862761,0.019464,0.178579,0.002403,12.643550,0.002598,12.644707,0.002576,dailymodel0
5,2.460681e+06,2.460681e+06,0.016067,-12.130043,4.233394,NaN,NaN,7.815653,19.082133,299.825572,...,0.008417,0.871790,0.048111,0.182076,0.005145,12.614045,0.004906,12.616045,0.004879,dailymodel0
6,2.460681e+06,2.460681e+06,-0.008045,6.774739,2.815522,NaN,NaN,12.345267,27.526787,299.765262,...,0.005170,0.831517,0.030800,0.176577,0.003514,12.632754,0.003691,12.633343,0.003639,dailymodel0
7,2.460686e+06,2.460686e+06,0.040945,4.610357,5.800396,NaN,NaN,5.388136,14.032025,299.821304,...,0.012453,0.891774,0.073522,0.177412,0.007154,12.639205,0.006070,12.639453,0.006092,dailymodel0
8,2.460726e+06,2.460726e+06,0.058585,-1.362416,2.957234,NaN,NaN,11.574093,26.713125,299.789961,...,0.005486,0.868275,0.031035,0.179768,0.003662,12.624598,0.003848,12.625768,0.003802,dailymodel0
9,2.460726e+06,2.460726e+06,-0.012008,-6.294036,2.703145,NaN,NaN,12.760667,28.473569,300.003695,...,0.004934,0.896557,0.028888,0.174780,0.003440,12.617792,0.003598,12.619572,0.003559,dailymodel0


In [4]:
#We take the timestamps (BJD) and the RV Errors from the dataframe.
times = df["bjd"]
rv_err = df["e_rv"]

print(f"Timestamps: \n{times}")
print(f"\nRV Errors: \n{rv_err}")

Timestamps: 
0     2.460662e+06
1     2.460662e+06
2     2.460664e+06
3     2.460665e+06
4     2.460666e+06
5     2.460681e+06
6     2.460681e+06
7     2.460686e+06
8     2.460726e+06
9     2.460726e+06
10    2.460726e+06
11    2.460726e+06
Name: bjd, dtype: float64

RV Errors: 
0     2.618923
1     3.141299
2     2.359089
3     1.850221
4     1.807651
5     4.233394
6     2.815522
7     5.800396
8     2.957234
9     2.703145
10    5.567126
11    5.375776
Name: e_rv, dtype: float64


In [5]:
#To define a Keplerian Model, we can use Radvel.
import radvel

In [6]:
#We define the orbital parameters of our system.
    #Period (P): Median value taken from the TESS observations
    #T0 (T0): Median value taken from the TESS observations.
    #Eccentricity: Kept to zero, assuming circular orbit.
    #Argument of Periapsis (omega): Kept as 90 degrees, but since radvel wants it in radians, π/2.
    #Semi Amplitude of RV (K): Kept as any semi-amplitude value you want to generate RV points for.
P = 0.2241957793
T0 = 2460258.8685502028
ecc = 0
omega = np.pi/2
K = 0

In [7]:
#Now we call in the Keplerian Model from radvel.
help(radvel.kepler.rv_drive)

Help on function rv_drive in module radvel.kepler:

rv_drive(t, orbel, use_c_kepler_solver=True)
    RV Drive
    Args:
        t (array of floats): times of observations
        orbel (array of floats): [per, tp, e, om, K].            Omega is expected to be            in radians
        use_c_kepler_solver (bool): (default: True) If             True use the Kepler solver written in C, else             use the Python/NumPy version.
    Returns:
        rv: (array of floats): radial velocity model



In [8]:
#We create the synthetic RV datapoints for given timestamps.
RV_data = radvel.kepler.rv_drive(times, [P, T0, ecc, omega, K], use_c_kepler_solver = True)

RV_data

0     0.0
1    -0.0
2    -0.0
3     0.0
4    -0.0
5    -0.0
6     0.0
7     0.0
8     0.0
9     0.0
10   -0.0
11   -0.0
Name: bjd, dtype: float64

In [9]:
#Then we add the RV error of our instrument to include the Errors we would expect if it were real data.
for i in range(len(RV_data)):
    RV_data[i] = RV_data[i] + np.random.normal(0, rv_err[i], 1)

RV_data

0    -3.643791
1     0.587158
2    -0.627313
3     0.430945
4    -0.858571
5     6.922054
6     0.380686
7    -7.487046
8    -4.092297
9     4.659152
10   -5.868608
11   -4.843392
Name: bjd, dtype: float64

In [10]:
#Now that we have generated our synthetic RV data, we have to turn it into CSV to use it for a Joint Fit.
#For that, first we create a DataFrame that will contain our data.
csv_data = pd.DataFrame({"bjd" : times,
                         "rv" : RV_data,
                         "e_rv" : rv_err})

csv_data

,bjd,rv,e_rv
0,2.460662e+06,-3.643791,2.618923
1,2.460662e+06,0.587158,3.141299
2,2.460664e+06,-0.627313,2.359089
3,2.460665e+06,0.430945,1.850221
4,2.460666e+06,-0.858571,1.807651
5,2.460681e+06,6.922054,4.233394
6,2.460681e+06,0.380686,2.815522
7,2.460686e+06,-7.487046,5.800396
8,2.460726e+06,-4.092297,2.957234
9,2.460726e+06,4.659152,2.703145


In [11]:
#We then save the DataFrame to an CSV File.
csv_data.to_csv(f"synthetic_rv_data_K_{K}.csv", index = False)

print("CSV File has been created!")

CSV File has been created!
